In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.cuda.amp import autocast, GradScaler
from torch.utils.data import DataLoader, ConcatDataset, Subset
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, confusion_matrix
from sklearn.model_selection import StratifiedKFold
from torchvision import transforms
import torch.nn.functional as F
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import timm
import os
import xml.etree.ElementTree as ET
from torch.utils.data import Dataset
from PIL import Image
import pickle
import random
from collections import Counter

class SurgicalImagesDataset(Dataset):
    def __init__(self, annotations_file, image_dir, augmented_image_dir=None, transform=None):
        self.annotations = self.parse_annotations(annotations_file)
        self.image_dir = image_dir
        self.augmented_image_dir = augmented_image_dir or "augmented_images"
        self.transform = transform

        # If augmented images directory is provided, assign class 2 to all images there
        self.augmented_images = []
        if augmented_image_dir:
            for fname in os.listdir(augmented_image_dir):
                if fname.endswith(('.jpg', '.png', '.jpeg')):  # Check for image files
                    self.augmented_images.append({'name': fname, 'label': 2})

    def parse_annotations(self, xml_file):
        tree = ET.parse(xml_file)
        root = tree.getroot()
        images = []
        labels = {}

        for label_elem in root.findall('.//label'):
            label_name = label_elem.find('name').text
            labels[label_name] = len(labels)

        for image_elem in root.findall('.//image'):
            tag_elem = image_elem.find('tag')
            if tag_elem is None:
                continue

            label_name = tag_elem.get('label')
            if label_name not in labels:
                continue

            image_data = {
                'name': image_elem.get('name'),
                'label': labels[label_name]
            }
            images.append(image_data)
        return images

    def __len__(self):
        return len(self.annotations) + len(self.augmented_images)

    def __getitem__(self, idx):
        if idx < len(self.annotations):
            image_data = self.annotations[idx]
            image_name = image_data['name']
            label = image_data['label']
            image_path = os.path.join(self.image_dir, image_name)
        else:
            image_data = self.augmented_images[idx - len(self.annotations)]
            image_name = image_data['name']
            label = image_data['label']
            image_path = os.path.join(self.augmented_image_dir, image_name)

        # Load the image
        image = Image.open(image_path).convert('RGB')

        # Apply transformations if any
        if self.transform:
            image = self.transform(image)

        return image, label

class FocalLoss(nn.Module):
    def __init__(self, alpha=1, gamma=2, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, inputs, targets):
        BCE_loss = nn.CrossEntropyLoss(reduction='none')(inputs, targets)
        pt = torch.exp(-BCE_loss)
        F_loss = self.alpha * (1 - pt) ** self.gamma * BCE_loss

        if self.reduction == 'mean':
            return torch.mean(F_loss)
        elif self.reduction == 'sum':
            return torch.sum(F_loss)
        else:
            return F_loss

class TransformSubset(Dataset):
    def __init__(self, subset, transform=None):
        self.subset = subset
        self.transform = transform

    def __len__(self):
        return len(self.subset)

    def __getitem__(self, idx):
        image, label = self.subset[idx]
        if self.transform:
            image = self.transform(image)
        return image, label

# Paths to annotations and images
annotations_files = ['/path_to_annotations1.xml',
                     '/path_to_annotations2.xml',
                     '/path_to_annotations3.xml']
image_dirs = ['/path_to_labelled_dataset1/images',
              '/path_to_labelled_dataset2/images',
              '/path_to_labelled_dataset3/images']

augmented_images_dir = '/path_to_augmented_frames_/highresolution_frames'

# Load datasets without augmentation
datasets = [SurgicalImagesDataset(ann_file, image_dir, augmented_image_dir=augmented_images_dir)
            for ann_file, image_dir in zip(annotations_files, image_dirs)]

combined_dataset = ConcatDataset(datasets)

# Augmentation transforms for training
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Transforms for validation
val_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Extract targets for stratified splitting
def extract_targets_from_concat_dataset(concat_dataset):
    all_targets = []
    for dataset in concat_dataset.datasets:
        all_targets.extend([annotation['label'] for annotation in dataset.annotations])
    return torch.tensor(all_targets)

all_targets = extract_targets_from_concat_dataset(combined_dataset)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Define SimCLRHead for classification, matching the pre-training architecture
class SimCLRHead(nn.Module):
    def __init__(self, in_dim, out_dim, hidden_dim=2048):
        super(SimCLRHead, self).__init__()
        self.mlp = nn.Sequential(
            nn.Linear(in_dim, hidden_dim),  # Increase dimensionality to hidden_dim (e.g., 2048)
            nn.ReLU(),
            nn.Linear(hidden_dim, out_dim),  # Reduce to the desired output dimension (e.g., 128)
        )

    def forward(self, x):
        x = self.mlp(x)
        return F.normalize(x, dim=-1)

    base_model = timm.create_model('tf_efficientnetv2_s', pretrained=True)
    num_ftrs = base_model.classifier.in_features
    base_model.classifier = nn.Identity()  # Remove the classifier layer

    # Define the SimCLR model (base model + SimCLR head)
    simclr_head = SimCLRHead(num_ftrs, 128, hidden_dim=2048)  # Match the hidden_dim used during pretraining
    simclr_model = nn.Sequential(base_model, simclr_head)

    # Load the pre-trained SimCLR weights
    simclr_model.load_state_dict(torch.load('/path_to_load/pretrained_contrastive_weights_4.pth', map_location=torch.device('cpu')))
    # Now that the weights are loaded, we can remove the SimCLR head and use the base model
    base_model = simclr_model[0]  # Extract the base model without the SimCLR head

    def create_classifier():
        classifier = nn.Sequential(
            base_model,          # Use the pre-trained EfficientNetV2-S model with loaded weights
            nn.Flatten(),        # Flatten the output from the base model
            nn.Linear(num_ftrs, 128),  # Add a fully connected layer
            nn.ReLU(),           # Add ReLU activation for non-linearity
            nn.Dropout(p=0.5),   # Add dropout with a 50% probability
            nn.Linear(128, 3)    # Final layer for classification
        )
        return classifier

# K-Fold Cross-Validation setup
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
results_path = '/path_to_save_/saved_fold_metrics.pkl'

# Initialize results storage
fold_results = []
fold_train_losses = []
fold_val_losses = []
fold_train_accuracies = []
fold_val_accuracies = []
fold_precisions = []
fold_recalls = []
fold_f1_scores = []
fold_accuracies = []
confusion_matrices = []
all_fold_predictions = []
all_fold_true_labels = []
all_fold_probs = []

# Paths for saving the best models and checkpoints
checkpoint_dir = '/path_to_save_foldwise_/checkpoints_/'

# Ensure checkpoint directory exists
if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)

# Track the current fold and model index
current_fold = 0

# Save/load fold and model index
fold_model_state_path = 'path_to_save/fold_model_state.pkl'

# Training loop with cross-validation
num_epochs = 30
patience = 3

scaler = GradScaler()

def save_checkpoint(state, filename='checkpoint.pth'):
    torch.save(state, filename)

def load_checkpoint(filename):
    checkpoint = torch.load(filename)
    return checkpoint

for fold, (train_val_idx, _) in enumerate(kf.split(torch.zeros(len(all_targets)), all_targets)):
    # Split the train_val_idx further into train and validation indices
    train_val_subset = Subset(combined_dataset, train_val_idx)

    # Split train_val_subset into training and validation sets
    train_size = int(0.8 * len(train_val_subset))
    val_size = len(train_val_subset) - train_size

    # Create Subsets
    train_indices, val_indices = torch.utils.data.random_split(list(range(len(train_val_subset))), [train_size, val_size])

    # Create Subsets
    train_subset = Subset(train_val_subset, train_indices)
    val_subset = Subset(train_val_subset, val_indices)

    # Initialize the criterion with dynamic class weights
    # Define the criterion with FocalLoss
    criterion = FocalLoss(alpha=1, gamma=2)  # You can adjust alpha and gamma based on your needs


    # Apply the transforms using TransformSubset
    train_dataloader = DataLoader(TransformSubset(train_subset, transform=train_transforms), batch_size=128, shuffle=True, num_workers=2)
    val_dataloader = DataLoader(TransformSubset(val_subset, transform=val_transforms), batch_size=128, shuffle=False, num_workers=2)

    # Instantiate the classifier model
    classifier = create_classifier().to(device)

    optimizer = optim.Adam(classifier.parameters(), lr=0.0001, weight_decay=1e-4)  # Adjust weight_decay
    start_epoch = 0
    best_loss = float('inf')
    best_val_acc = 0.0
    epochs_no_improve = 0
    train_losses = []
    val_losses = []
    train_accuracies = []
    val_accuracies = []

    # Initialize the learning rate scheduler
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3)

    for epoch in range(start_epoch, num_epochs):
        if epochs_no_improve >= patience:
            print(f"Early stopping at epoch {epoch+1}")
            break

        classifier.train()
        running_loss = 0.0
        correct = 0
        total = 0

        progress_bar = tqdm(enumerate(train_dataloader), total=len(train_dataloader), desc=f"Epoch [{epoch+1}/{num_epochs}]")
        for batch_idx, (images, labels) in progress_bar:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            with autocast():  # Mixed precision training
                outputs = classifier(images)
                loss = criterion(outputs, labels)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            # Update progress bar with loss and accuracy
            progress_bar.set_postfix(loss=loss.item(), accuracy=100 * correct / total)

        epoch_loss = running_loss / len(train_dataloader)
        epoch_acc = 100 * correct / total

        train_losses.append(epoch_loss)
        train_accuracies.append(epoch_acc)

        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.2f}%")

        # Validation
        classifier.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0
        all_preds = []
        all_labels = []

        with torch.no_grad():
            for images, labels in val_dataloader:
                images, labels = images.to(device), labels.to(device)
                outputs = classifier(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()
                all_preds.extend(predicted.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

        val_loss /= len(val_dataloader)
        val_acc = 100 * val_correct / val_total
        val_losses.append(val_loss)
        val_accuracies.append(val_acc)
        print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_acc:.2f}%")

        # Save the model with the best validation loss
        if val_loss < best_loss:
            best_loss = val_loss
            torch.save(classifier.state_dict(), os.path.join(checkpoint_dir, f'/path_to_save_foldwise_/best_model_fold{fold+1}_val_loss.pth'))
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1

        # Save the model with the best validation accuracy
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(classifier.state_dict(), os.path.join(checkpoint_dir, f'/path_to_save_foldwise_/best_model_fold{fold+1}_val_acc.pth'))

        # Save checkpoint
        save_checkpoint({
            'epoch': epoch,
            'model_state_dict': classifier.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'best_loss': best_loss,
            'best_val_acc': best_val_acc,
            'epochs_no_improve': epochs_no_improve,
            'train_losses': train_losses,
            'val_losses': val_losses,
            'train_accuracies': train_accuracies,
            'val_accuracies': val_accuracies
        }, filename=os.path.join(checkpoint_dir, f'checkpoint_fold{fold+1}.pth'))

    # Evaluate on validation set (used as test here)
    classifier.eval()
    val_correct = 0
    val_total = 0
    all_preds = []
    all_probs = []
    all_val_labels = []

    with torch.no_grad():
        for images, labels in val_dataloader:
            images, labels = images.to(device), labels.to(device)
            outputs = classifier(images)
            _, predicted = torch.max(outputs.data, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()
            all_preds.extend(predicted.cpu().numpy())
            all_probs.extend(outputs.cpu().numpy())  # Save probabilities
            all_val_labels.extend(labels.cpu().numpy())

    # Save predictions, probabilities, and true labels for this fold
    all_fold_predictions.append(all_preds)
    all_fold_true_labels.append(all_val_labels)
    all_fold_probs.append(all_probs)

    val_acc = 100 * val_correct / val_total
    fold_accuracies.append(val_acc)

    # Calculate precision, recall, f1-score for each class on validation set
    precision, recall, f1, _ = precision_recall_fscore_support(all_val_labels, all_preds, average=None)
    accuracy = accuracy_score(all_val_labels, all_preds)

    fold_precisions.append(precision)
    fold_recalls.append(recall)
    fold_f1_scores.append(f1)

    fold_results.append({'fold': fold + 1, 'val_loss': best_loss, 'val_acc': best_val_acc, 'test_acc': val_acc})
    fold_train_losses.append(train_losses)
    fold_val_losses.append(val_losses)
    fold_train_accuracies.append(train_accuracies)
    fold_val_accuracies.append(val_accuracies)

    # Compute and store confusion matrix
    cm = confusion_matrix(all_val_labels, all_preds)
    confusion_matrices.append(cm)

    # Save results after each fold
    with open(results_path, 'wb') as f:
        pickle.dump({
            'fold_results': fold_results,
            'fold_train_losses': fold_train_losses,
            'fold_val_losses': fold_val_losses,
            'fold_train_accuracies': fold_train_accuracies,
            'fold_val_accuracies': fold_val_accuracies,
            'fold_precisions': fold_precisions,
            'fold_recalls': fold_recalls,
            'fold_f1_scores': fold_f1_scores,
            'fold_accuracies': fold_accuracies,
            'confusion_matrices': confusion_matrices,
            'all_fold_predictions': all_fold_predictions,
            'all_fold_true_labels': all_fold_true_labels,
            'all_fold_probs': all_fold_probs
        }, f)

    # Update the fold and model index state
    with open(fold_model_state_path, 'wb') as f:
        pickle.dump((fold + 1, 0), f)  # Move to the next fold

# Save the fold-wise predictions, probabilities, and true labels using pickle
with open('/path_to_save_foldwise_predictions.pkl', 'wb') as f:
    pickle.dump(all_fold_predictions, f)
with open('/path_to_save_foldwise_truelabels.pkl', 'wb') as f:
    pickle.dump(all_fold_true_labels, f)
with open('/path_to_save_foldwise_probablities', 'wb') as f:
    pickle.dump(all_fold_probs, f)

# Print the results fold-wise
for i in range(len(fold_results)):
    print(f"Fold {fold_results[i]['fold']}:")
    print(f"  Validation Loss: {fold_results[i]['val_loss']:.4f}")
    print(f"  Validation Accuracy: {fold_results[i]['val_acc']:.2f}%")
    print(f"  Test Accuracy: {fold_results[i]['test_acc']:.2f}%")
    print(f"  Precision (%):")
    for j in range(len(fold_precisions[i])):
        print(f"    Class {j}: {fold_precisions[i][j]*100:.2f}%")
    print(f"  Recall (%):")
    for j in range(len(fold_recalls[i])):
        print(f"    Class {j}: {fold_recalls[i][j]*100:.2f}%")
    print(f"  F1 Score:")
    for j in range(len(fold_f1_scores[i])):
        print(f"    Class {j}: {fold_f1_scores[i][j]:.2f}")
    print()  # Empty line for separation between folds

# Plotting accuracy and loss curves for each fold
plt.figure(figsize=(20, 10))

# Plot accuracy curves
plt.subplot(2, 1, 1)
for i, (train_accuracies, val_accuracies) in enumerate(zip(fold_train_accuracies, fold_val_accuracies)):
    plt.plot(train_accuracies, label=f'Train Accuracy Fold {i+1}')
    plt.plot(val_accuracies, label=f'Val Accuracy Fold {i+1}')
plt.title('Accuracy Curves')
plt.xlabel('Epoch')
plt.ylabel('Accuracy (%)')
plt.legend()

# Plot loss curves
plt.subplot(2, 1, 2)
for i, (train_losses, val_losses) in enumerate(zip(fold_train_losses, fold_val_losses)):
    plt.plot(train_losses, label=f'Train Loss Fold {i+1}')
    plt.plot(val_losses, label=f'Val Loss Fold {i+1}')
plt.title('Loss Curves')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()

# Plot confusion matrices
for i, cm in enumerate(confusion_matrices):
    plt.figure(figsize=(10, 7))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title(f'Confusion Matrix for Fold {i+1}')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.show()
